# Klassifiziere Verkehrsschilder

In diesem Jupyter Notebook geht es um die Erkennung von Verkehrszeichen.
Der dafür verwendete Datensatz ist der German Traffic Sign Recognition Benchmark, kurz GTSRB.
Sämtliche Informationen rund um den Datensatz sind [online](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) einsehbar.

In [ ]:
import numpy as np
import os.path
import imageio
import skimage.transform
import skimage.feature
import skimage.color
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

Außerdem gibt es noch ein selbstgeschriebenes Modul, welches quasi neben den Jupyter Notebooks liegt.
Hier sind Funktionen ausgelagert worden, die das Notebook unübersichtlich machen würden.

In [ ]:
import gtsrb_image_database_loader

Das obige Modul kann die Bilddatenbank so laden, dass alle Metadaten vorliegen.
Nur so kann man später die Bilder für die Erkennung einsetzen.
Allerdings müssen die Bilder auf der Festplatte liegen.
Der Schritt des Herunterladens muss händisch vorgenommen werden.

1. Für das Herunterladen der Daten für die Übung klicken Sie bitte [hier](https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip). Bitte laden Sie die ZIP-Datei herunter.
2. Entpacken Sie das ZIP-Archiv in den gleichen Ordner wie dieses Jupyter Notebook.

Falls man sich auf einem PC mit Linux als Betriebssystem befindet, kann man auch das Skript `download_traffic_signs.sh` ausführen, welches sich im gleichen Ordner wie dieses Jupyter Notebook befindet.



Laden der Meta-Informationen
--------------------------------

Zunächst laden wir die Meta-Informationen, d. h. welche Bilder es gibt, wie groß diese sind, wo diese auf der Festplatte liegen sowie in welchem Ausschnitt des Bildes sich das Verkehrszeichen befindet.
Legen Sie das Verzeichnis `GTSRB` in das gleiche Verzeichnis wie das Notebook.
Falls das Kopieren zu viel Zeit benötigt, passen Sie `path_to_directory` so an, dass der Pfad auf den entpackten Ordner mit den Verkehrsschildern zeigt.

In [ ]:
# starte im gleichen Ordner (relativer Pfad)
path_to_directory = "./GTSRB_Final_Training_Images/GTSRB/Final_Training/Images"  

# alternativ: gebe absoluten Pfad an
#path_to_directory = "C:/asdfasdf/"

df = gtsrb_image_database_loader.load_traffic_sign_database(path_to_directory)

df.head()

Laden der Bilder
-------------------

Hier werden alle Bilder geladen, zurechtgeschnitten und auf eine einheitliche Größe gebracht.
Farbinformationen können vernachlässigt werden.
Die Bilder werden dann dem DataFrame als neue Spalte hinzugefügt.

In [ ]:
images = []
for row in gtsrb_image_database_loader.log_progress(df.itertuples(), size=len(df)):
    color_image = imageio.imread(row.path_to_image).astype(int, copy=False)
    gray_image = skimage.color.rgb2gray(color_image)
    cropped_image = gray_image[row.Roi_Y1:row.Roi_Y2, row.Roi_X1:row.Roi_X2]
    resized_image = skimage.transform.resize(cropped_image, [40, 40], mode="constant")
    images.append(resized_image)
df = df.assign(image=images)

Ein Verkehrsschild sieht nun so aus:

In [ ]:
sample_image = df.iloc[0].image
plt.imshow(sample_image, cmap='gray')
plt.show()

Extrahieren eines Features
-------------------------------

Mithilfe von Histograms of Gradients (HOGs) können Gegenstände gut automatisiert klassifiziert werden.
Diese werden z. B. im Artikel ["Histograms of Oriented Gradients for Human Detection" von N. Dalal und B. Triggs](https://hal.inria.fr/docs/00/54/85/12/PDF/hog_cvpr2005.pdf) gut beschrieben.
Die HOGs des oben gezeigte Beispielbild werden im Folgenden angezeigt.

Eine einfache Möglichkeit, Bilder in HOGs umzuwandeln, wird von [scikit-image](http://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.hog) bereitgestellt und im Folgenden verwendet.
Die Parameter `orientations`, `pixels_per_cell`, `cells_per_block` und `block_norm` beziehen sich auf den HOG-Algorithmus und können variiert werden.

Zunächst wird für das obige Verkehrszeichen gezeigt, wie die HOG-Repräsentation aussieht.

In [ ]:
hog_img = skimage.feature.hog(
    sample_image,
    transform_sqrt=True,
    orientations=8,
    pixels_per_cell=(6, 6),
    cells_per_block=(3, 3),
    feature_vector=True,
    block_norm="L2-Hys",
    visualize=True
)[1]
plt.imshow(hog_img)
plt.show()

Nun werden die HOG-Werte für sämtliche Bilder berechnet.
Das Ergebnis wird als eine neue Spalte dem DataFrame hinzugefügt.

In [ ]:
hog_features = []
for row in gtsrb_image_database_loader.log_progress(df.itertuples(), size=len(df)):
    hog_feature = skimage.feature.hog(
        row.image,
        transform_sqrt=True,
        orientations=8,
        pixels_per_cell=(6, 6),
        cells_per_block=(3, 3),
        feature_vector=True,
        block_norm="L2-Hys",
        visualize=False
    )
    hog_features.append(hog_feature)
df = df.assign(hog_feature=hog_features)

Nun sind die Daten soweit vorbereitet, dass der Zusammenhang zwischen dem Bild und dem Verkehrszeichen durch eine geeignete Methode des ML hergestellt werden kann.

Aufteilen in Trainings- und Validierungs-Sets
------------------------------------------------------

Es werden 67 % als Trainings- und 33 % als Test-Daten verwendet.

In [ ]:
features = df["hog_feature"].values
targets = df["ClassId"].values

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.33)

Noch eine Kleinigkeit:
Die HOGs liegen als Arrays von Arrays vor, der Lernalgorithmus benötigt aber ein einzelnes langes Array.
Dafür müssen die Arrays aneinander gehängt werden.
Dies kann einfach durch[`np.stack`](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.stack.html) vorgenommen werden.

In [ ]:
X_train = np.stack(X_train)
X_test = np.stack(X_test)

Training und Evaluierung mit einem Klassifizierer
--------------------------------------------------------

Als Klassifizierungs-Algorithmus wird ein Random Forest (RF) eingesetzt.
Der Random Forest ist ein Ensemble von mehreren Entscheidungsbäumen.
Im Folgenden wird eine [Implementierung von scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) eingesetzt.

Für die Bewertung der Ergebnisse können nun verschiedene Aspekte betrachtet werden.
Die Accuracy ist ein einfaches Maß, um den Anteil der richtig bestimmten Klassen zu quantifizieren.
Die Confusion Matrix ermöglicht es, zu sehen, welche Klassen miteinander verwechselt werden.

In [ ]:
random_forest = RandomForestClassifier(
    n_estimators=10,
    max_depth=7,
    n_jobs=-1,
    max_features="auto",
    class_weight=None
)
random_forest.fit(X_train, y_train)
predicted = random_forest.predict(X_test)
confusion_matrix = metrics.confusion_matrix(
    y_test,
    predicted,
    labels=range(43)
)
score = random_forest.score(X_test, y_test)
print("Accuracy: {score} %".format(score=score*100))

Eine schöne Möglichkeit, eine Confusion Matrix zu visualisieren, ist, diese als Bild zu plotten.
Der folgende Code ist der [Dokumentation von scikit-learn](http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html) entnommen und angepasst worden.
Die Verkehrsschilder (als Liste in der Variablen `classes`) werden durch die Zahlen 0 bis 42 repräsentiert.

In [ ]:
classes = df.ClassId.unique()

normalized_confusion_matrix = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis]

fig, ax = plt.subplots(figsize=(20, 20))
img = ax.imshow(normalized_confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
    
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45, label='Tatsächliche Klassen-ID')
plt.yticks(tick_marks, classes, label='Vom Algorithmus angenommene Klassen-ID')

plt.title("Confusion Matrix")
plt.colorbar(img, shrink=0.2)
plt.show()

### Mit interaktiven Widgets die Datenbank durchsuchen

Hier kann man nun sehen, welche Verkehrszeichen besonders häufig miteinander verwechselt werden.
Aber welche werden denn nun verwechselt?
Dafür gibt es nun ein kleines Such-Tool.

In [ ]:
def plot_sample_for_class_id(_id):
    sample_image_for_class_id = df[df.ClassId == _id].iloc[0]
    plt.title("Class ID: " + str(int(_id)))
    plt.imshow(sample_image_for_class_id.image, cmap='gray')
    return sample_image_for_class_id

plot_sample_for_class_id(0)

In [ ]:
import ipywidgets as widgets
from IPython.display import clear_output

input_field = widgets.FloatText()

def on_change(x):
    clear_output(wait=True)
    display(input_field)
    information = plot_sample_for_class_id(x["new"])
    display(information)
    
input_field.observe(on_change, names='value')

input_field

Dieser Datensatz ist bereits häufiger in [wissenschaftlichen Veröffentlichungen](http://benchmark.ini.rub.de/?section=gtsrb&subsection=results) referenziert worden.
Besonders zu empfehlen ist der Artikel "Man vs. computer: Benchmarking machine learning algorithms for traffic sign recognition" von J. Stallkamp, M. Schlipsing, J. Salmena und C. Igel.

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0; display:inline" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a> &nbsp;&nbsp;&nbsp;&nbsp;Dieses Werk von Marvin Kastner ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Namensnennung 4.0 International Lizenz</a>.